In [ ]:
#importing all nececarry modules
from collections import deque
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random

In [ ]:
##defining game environment
class tic_tac_toe:
    def __init__(self):
        self.state = [0,0,0,0,0,0,0,0,0]
        self.input_size = 9
        self.output_size = 9
        self.player1=True
        self.move=0
        self.last_move = 0

    def done(self,filled):
        if (filled[0] == filled[1]== filled[2]!=0):
             return True
        elif (filled[0] == filled[3]== filled[6]!=0):
            return True
        elif (filled[0] == filled[4]== filled[8]!=0):
            return True
        elif (filled[1] == filled[4]== filled[7]!=0):
            return True
        elif (filled[2] == filled[4]== filled[6]!=0):
            return True
        elif (filled[2] == filled[5]== filled[8]!=0):
            return True
        elif (filled[3] == filled[4]== filled[5]!=0):
            return True
        elif (filled[6] == filled[7]== filled[8]!=0):
            return True
        elif self.move == 9:
            return True
        return False


    def reset(self):
        self.state = [0,0,0,0,0,0,0,0,0]
        self.player1=True
        self.move=0
        self.last_move = 0

    def validate_move(self,move):
        if(0<=move<10):
            return(True) if (self.state[move]==0) else False
        return(False)


    def cell_move(self,move):
        if(self.validate_move(move)):
            self.move = self.move + 1
            self.state[move] = 1 if(self.player1) else 2
            self.player1 = not self.player1
            return True
        else:
            return False
    
    def step(self,move):
        state= np.copy(self.state)
        reward = self.reward(state,move)
        self.move = self.move+1
        self.last_move = move+1
        self.state[move] = 1 if(self.player1) else 2
        
        next_state = self.state
        done = self.done(next_state)
        self.player1 = not self.player1
        return reward,next_state,done


    def reward(self,state,move):
        local_state = np.copy(state)
        local_state[move] = 1 if(self.player1) else 2
        if(self.done(local_state)):
            return 40
        reward = self.missed_win(state,move) + self.accepted_loss(state,move) + self.blocked_win(state,move)
        return reward
    
    def missed_win(self,state,move):
        local_copy = np.copy(state)
        for cell in range(9):
            if(local_copy[cell] == 0 and not(cell == move) ):
                local_copy[cell] = 1 if(self.player1) else 2
                if(self.done(local_copy)):
                    local_copy[cell] = 0 
                    print("missed win -20",move,"\n",np.reshape(local_copy,(3,3)))
                    return -20
                else:
                    local_copy[cell] = 0
        return 0

    def accepted_loss(self,state,move):
        local_copy = np.copy(state)
        for cell in range(9):
            if(local_copy[cell]==0 and not(cell == move) ):
                local_copy[cell] = 1 if(not self.player1) else 2
                if(self.done(local_copy)):
                    local_copy[cell] = 0 
                    print("accepted loss -20",move,"\n",np.reshape(local_copy,(3,3)))
                    return -20
                else:
                    local_copy[cell] = 0
        return 0

    def blocked_win(self,state,move):
        local_copy = np.copy(state)
        local_copy[move] = 1 if(not self.player1) else 2
        if(self.done(local_copy)):
            local_copy[move] = 0 
            print("blocked win 20",move,"\n",np.reshape(local_copy,(3,3)))
            return 20
        return 0

    def print_game_state(self):
        print("player2" if self.player1 else "player1",self.last_move)
        print(np.reshape(np.copy(self.state),(3,3)))

In [ ]:
##defining agent
class agent:
    def __init__(self,active_game):
        self.active_game=active_game

        self.input_size = self.active_game.input_size
        self.output_size=self.active_game.output_size
        
        self.memory = deque(maxlen = 100)

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.01
        self.model = self.create_model()
    

    def create_model(self):
        model = Sequential()

        model.add(Dense(22,input_dim=self.active_game.input_size,activation='relu'))
        model.add(Dense(11,activation='relu'))
        model.add(Dense(self.active_game.output_size,activation='softmax'))

        model.compile(loss='mse',optimizer=Adam(learning_rate=self.learning_rate))

        return model


    def play(self,i):
        return(self.active_game.cell_move(i))
    
    def act(self):
        local_copy = np.copy(self.active_game.state)
        allowed_moves = np.where(local_copy==0)[0]
        
        if np.random.rand()<self.epsilon:
            return np.random.choice(allowed_moves,1)[0]
        
        local_copy = np.reshape(np.copy(local_copy),(1,9))
        prediction = self.model.predict(local_copy)[0]
        predicted_action = np.where(prediction == np.amax(prediction))[0][0]
        
        
        return  predicted_action


    def remember(self,player1,state,action,reward,next_state,done):
        self.memory.append((player1,state,action,reward,next_state,done))
    
    def short_train(self,state,action,reward):
        local_state = np.reshape(np.copy(state),(1,9))

        target_f = np.array(self.model.predict(local_state))
        target_f[0][action] = reward

        self.model.fit(local_state,target_f,epochs=1,verbose=0)

    def train(self,batch_size=15):
        mini_batch = random.sample(self.memory,batch_size)

        for exprience in mini_batch:
            player1,state,action,reward,next_state,done = exprience

            local_state = np.reshape(np.copy(next_state),(1,9))

            if not done:
                target = reward + self.gamma*np.amax(self.model.predict(local_state)[0])
            else:
                target = reward
            
            local_state = np.reshape(np.copy(state),(1,9))

            target_f = np.array(self.model.predict(local_state))
            target_f[0][action] = target

            self.model.fit(local_state,target_f,epochs=1,verbose=0)

        if(self.epsilon>self.epsilon_min):
            self.epsilon *= self.epsilon_decay

    def save(self,name):
        self.model.save_weights(name)
    def load(self,name):
        self.model.load_weights(name)

In [ ]:
import numpy as np

active_game = tic_tac_toe()
player=agent(active_game)
batch_size=15
outputdir_ = "tic_tac_toe_model"

play=True

for episodes in range(10):
    active_game.reset()
    print(episodes,'episode')
    while play:

        state = active_game.state

        while True:
            action = player.act()
            if(active_game.validate_move(action)):
                break
            else:
                next_state = np.copy(state)
                next_state[action] = 1 if(active_game.player1) else 2
                player.remember(active_game.player1,state,action,-100,next_state,False)
                player.short_train(state,action,-100)

  
        reward,next_state,done =  active_game.step(action)

        player.remember(active_game.player1,state,action,reward,next_state,done)
        
        if(episodes>8):
            print("move",active_game.move)
            print(active_game.print_game_state())
            print("------------------------")
        

        if done:
            if len(player.memory) > batch_size:
                player.train(batch_size)
            if (episodes+1) % 10 == 0:
                player.save(outputdir_+"weights"+'{:04d}'.format(episodes+1)+".hdf5")
            print("player2") if(active_game.player1) else print("draw") if (active_game.move == 9 and not active_game.done(next_state)) else print("player1")
            break
    print('') 
            
print("all done")

In [ ]:
# playing game with trained model
# calling certain episode number trained model
episode_number = 10
active_game = tic_tac_toe()
player = agent(active_game)
player.load("tic_tac_toe_modelweights"+'{:04d}'.format(episode_number)+".hdf5")
print(player.model)

# specific state for which the trained model is to be checked
state = [[1,0,1],
         [2,2,1],
         [2,1,2]]
local_copy = np.reshape(state,(1,9))
print(local_copy)
prediction = player.model.predict(local_copy)[0]
predicted_action = np.where(prediction == np.amax(prediction))[0][0]
print(prediction,predicted_action)